In [50]:
#########################################################################################################
# PACKAGES

import dash
from dash import dcc, html, callback
import plotly.graph_objects as go
import pandas as pd
from dash.dependencies import Input, Output
import plotly.colors
from datetime import datetime



#########################################################################################################
# DATA

# Loading noise and weather data
data_noise = pd.read_csv('../Data for visualization/daily_noisedata_2022.csv', header=0, sep=',')
data_weather = pd.read_csv('../Data for visualization/daily_weatherdata_2022.csv', header=0, sep=',')

# Calculate mean noise level per day
noise_per_day = data_noise.groupby(['day', 'month'])['laeq'].mean().reset_index()

# Calculate mean temperature per day
temp_per_day = data_weather.groupby(['Day', 'Month'])['LC_TEMP_QCL3'].mean().reset_index()

# Merge into 1 dataframe
merged_df = noise_per_day.merge(temp_per_day, left_on=['day', 'month'], right_on=['Day', 'Month'])
merged_df = merged_df.drop(['Day', 'Month'], axis=1)

# Divide temperature values into bins
merged_df['temp_bins'] = pd.cut(merged_df['LC_TEMP_QCL3'], bins=73) #take 5 observations per bin, i.e. 73 bins

# Calculate the mean temperature for each bin
temp_mean_per_bin = merged_df.groupby('temp_bins')['LC_TEMP_QCL3'].mean()

# Create a dictionary mapping bin labels to the mean temperature
temp_mean_dict = temp_mean_per_bin.to_dict()

# Update the temperature values with the mean temperature per bin
merged_df['LC_TEMP_QCL3'] = merged_df['temp_bins'].map(temp_mean_dict)

# Sort the dataframe based on the bins
merged_df = merged_df.sort_values('temp_bins')

# Remove the temp_bins column
merged_df = merged_df.drop('temp_bins', axis=1)

# Round temperatures to 1 unit because we just want a rough overview + order the dataframe by temp
#merged_df['LC_TEMP_QCL3'] = merged_df['LC_TEMP_QCL3'].round(0)
#merged_df = merged_df.sort_values('LC_TEMP_QCL3')

In [51]:
merged_df.head()

,day,month,laeq,LC_TEMP_QCL3
203,17,12,49.432297,-4.535916
179,15,12,50.744154,-3.928799
215,18,12,48.968711,-3.301763
155,13,12,49.258672,-2.918916
167,14,12,49.294234,-2.498080


In [54]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=merged_df['LC_TEMP_QCL3'],
    y=merged_df['laeq'],
    mode='lines',
    fill='tozeroy',
    line=dict(color='#E6AF2E'),
    fillcolor='rgba(230, 175, 46, 0.8)',
    name='Noise',
    hovertemplate='Temperature: %{x:.1f}°C<br>Noise Level: %{y:.2f} dB(A)',
    hoverlabel=dict(namelength=0),
))

fig.update_layout(
    title=dict(
        text='How does the average noise level vary with temperature?',
        x=0.5,
        font=dict(color="white", size=24)
    ),
    xaxis=dict(
        title='Temperature (°C)',
        showgrid=True,
        zeroline=False,
        gridcolor='rgba(255, 255, 255, 0.1)',
        title_font=dict(color="white", size=18),
        tickfont=dict(color='white')
    ),
    yaxis=dict(
        title='Average noise level (dB(A))',
        showgrid=True,
        zeroline=True,
        gridcolor='rgba(255, 255, 255, 0.1)',
        title_font=dict(color="white", size=18),
        range=[45, merged_df['laeq'].max()],
        tickfont=dict(color='white')
    ),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    hovermode='x',
    legend=dict(
        font=dict(color='white')
    )
)

fig.show()